<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/fcns/training_B11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## B1.6 radar + gauges 1 min + cmls -> gauges 1 min [xrlg1_yg1]
# mode leave one gauge out

In [ ]:
! git clone https://github.com/nanopiero/fusion.git

In [8]:
! pwd

/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation/fusion/notebooks/fcns


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import os
import time
import sys
sys.path.append('/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation')

from fusion.utils.datasets import spatialized_gt, create_cmls_filter, FusionDataset
from fusion.utils.datasets import indices_to_sampled_values, get_point_measurements, point_gt, segment_gt, make_noisy_images
from torch.utils.data import DataLoader
from fusion.utils.fcn import UNet
from fusion.utils.cost_functions import QPELoss_fcn, compute_metrics
from fusion.utils.viz import set_tensor_values2, plot_images, plot_images_10pts_20seg, plot_results_10pts_20seg

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Config
num_epochs = 2000
save_every = 2
path = r'/scratch/mdso/lepetitp/ppc/RAINCELL/models/simulation/checkpoint_fcn_exp_B16_xrlg1_yg1.pt'
npairs = 20
nsteps = 60
ndiscs = 5
size_image=64
length_dataset = 6400
device = torch.device('cuda:0')

# Entraînement
npoints = 20
dataset = FusionDataset(length_dataset=length_dataset,
                        npairs=npairs,
                        nsteps=nsteps,
                        ndiscs=ndiscs, size_image=size_image)


loader = DataLoader(dataset, batch_size=64, num_workers=4)


In [3]:
# Tiny UNet V1. 60 new channels for input time series of rain gauges measurements
use_fcn = True
ch_in = 72 + 60
ch_out = nsteps * 3 + 1
size = nsteps * 3

model = UNet(ch_in, ch_out, size, nb_additional_parameters=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = QPELoss_fcn()

In [4]:
# Boucle avec 5 modes d'évaluation
val_steps = ['eval_opportunity_cost_spat',
             # 'eval_added_value_few_spat',
             # 'eval_added_value_half_spat',
             'eval_added_value_full_spat',
             # 'eval_added_value_full_id',
             ]
steps = val_steps + ['train']
losses = {step:[] for step in steps}
last_epoch = 0

In [5]:
checkpoint = torch.load(path, \
                            map_location=device)
last_epoch = checkpoint['epoch']
losses = checkpoint['train_losses']
# best_loss = checkpoint['best_loss']
model_weights = checkpoint['model']
optimizer_state_dict = checkpoint['optimizer']
# scheduler_state_dict = checkpoint['scheduler']
model.load_state_dict(model_weights)
optimizer.load_state_dict(optimizer_state_dict)
# scheduler.load_state_dict(scheduler_state_dict)

In [6]:
last_epoch

54

In [ ]:
model.train()

for epoch in range(last_epoch, num_epochs + 1):
  t = time.time()
  print('epoch n°', epoch, '\n')

  running_regression_loss = {step:0.0 for step in steps}
  running_regression_loss_1h = {step:0.0 for step in steps}
  running_segmentation_loss = {step:0.0 for step in steps}
  running_confusion_matrix = {step: np.zeros((2, 2), dtype=int) for step in steps}

  for i, (images, pairs, filters) in enumerate(loader):

    # ground truth (not usable)
    images = images.clone().detach().float().to(device)

    # pseudo radar
    noisy_images = make_noisy_images(images)

    # pseudo CMLs
    pairs = pairs.clone().detach().float().to(device)
    filters = filters.clone().float().detach().to(device)

    # segment_measurements = segment_gt(images, pairs, filters)
    _, segment_measurements_fcn = segment_gt(images, pairs, filters,
                                             use_fcn=use_fcn)

    #Validation steps
    model.eval()
    with torch.no_grad():

      # splitting
      np_val_inputs_few = 1
      np_val_inputs_half = npoints//4  # 5
      np_val_inputs_comp = npoints//2 - np_val_inputs_half - np_val_inputs_few  # 4
      np_val_targets = npoints//2 # 10

      # val split
      split_val = [np_val_inputs_few, np_val_inputs_half, np_val_inputs_comp, np_val_targets]
      split_few, split_half, split_comp, split_targets = point_gt(images, npoints=npoints,
                                                                    use_fcn=use_fcn,
                                                                    split=split_val)
      _, point_measurements_fcn_eval_few, _ = split_few
      _, point_measurements_fcn_eval_half, _ = split_half
      _, point_measurements_fcn_eval_comp, _ = split_comp
      _, point_measurements_fcn_val_targets, _ = split_targets

      # 4 first val steps (10 last pluvios for testing generalization)
      targets = point_measurements_fcn_val_targets

      # val step 1 : eval_opportunity_cost_spat
      step = 'eval_opportunity_cost_spat'
      inputs = torch.cat([noisy_images,
                          segment_measurements_fcn,
                          0 * point_measurements_fcn_eval_few - 0.1
                          ], dim=1)
      outputs = model(inputs)
      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm

      del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss
      torch.cuda.empty_cache()


      step = 'eval_added_value_full_spat'
      point_measurements_fcn_eval_full = point_measurements_fcn_eval_few + \
                                          point_measurements_fcn_eval_half + \
                                          point_measurements_fcn_eval_comp

      point_measurements_fcn_eval_full += 2 * 0.1 * (point_measurements_fcn_eval_few >= 0)
      point_measurements_fcn_eval_full += 2 * 0.1 * (point_measurements_fcn_eval_half >= 0)
      point_measurements_fcn_eval_full += 2 * 0.1 * (point_measurements_fcn_eval_comp >= 0)

      point_measurements_fcn_eval_full[point_measurements_fcn_eval_full<0] = -0.1

      inputs = torch.cat([noisy_images,
                          segment_measurements_fcn,
                          point_measurements_fcn_eval_full
                          ], dim=1)

      outputs = model(inputs)
      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm

      del inputs, loss, regression_loss, regression_loss_1h, segmentation_loss, split_few, split_half, split_comp, split_targets
      torch.cuda.empty_cache()

      del point_measurements_fcn_eval_full
      torch.cuda.empty_cache()

    # train step
    model.train()
    step = 'train'
    optimizer.zero_grad()  # Zero the gradients
    # leave one gauge out :
    for k in range(0,10):
        if k == 0 : 
            ng_train_targets = 1
            ng_train_inputs = npoints // 2 - 1
            split_train = [ng_train_targets, ng_train_inputs]
            # split  train
            split_targets, split_inputs = point_gt(images, npoints=npoints,
                                                   use_fcn=use_fcn,
                                                   split=split_train)
        
            _, point_measurements_fcn_train_inputs, _ = split_inputs
            _, point_measurements_fcn_train_targets, _ = split_targets
            
        elif k == 9 : 
            ng_train_targets = 1
            ng_train_inputs = npoints // 2 - 1
            split_train = [ng_train_inputs, ng_train_targets]
            # split  train
            split_inputs, split_targets = point_gt(images, npoints=npoints,
                                                   use_fcn=use_fcn,
                                                   split=split_train)
        
            _, point_measurements_fcn_train_inputs, _ = split_inputs
            _, point_measurements_fcn_train_targets, _ = split_targets
        else : 
            ng_train_targets = 1
            ng_train_inputs0 = k - 1
            ng_train_inputs1 = npoints // 2 - k
            split_train = [ng_train_inputs0, ng_train_targets, ng_train_inputs1]
            
            # split  train
            split_inputs0, split_targets, split_inputs1 = point_gt(images, npoints=npoints,
                                                                    use_fcn=use_fcn,
                                                                    split=split_train)
            
            _, point_measurements_fcn_train_inputs0, _ = split_inputs0
            _, point_measurements_fcn_train_targets, _ = split_targets
            _, point_measurements_fcn_train_inputs1, _ = split_inputs1

            point_measurements_fcn_train_inputs = point_measurements_fcn_train_inputs0
            point_measurements_fcn_train_inputs[point_measurements_fcn_train_inputs1>=0] = \
                        point_measurements_fcn_train_inputs1[point_measurements_fcn_train_inputs1>=0]
      
        inputs0 = torch.cat([noisy_images,
                            segment_measurements_fcn,
                            point_measurements_fcn_train_inputs
                            ], dim=1)
          
        inputs1 = torch.cat([noisy_images,
                            segment_measurements_fcn,
                            point_measurements_fcn_train_targets
                            ], dim=1)
          
        targets0 = point_measurements_fcn_train_targets
        targets1 = point_measurements_fcn_train_inputs
          
 
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)      
        regression_loss, regression_loss_1h, segmentation_loss, loss0, batch_cm, _ = criterion(model.p, outputs0, targets0)
        _, _, _, loss1, _, _ = criterion(model.p, outputs1, targets1)
        loss = loss0 + loss1
        loss.backward()  # Backward pass

    
        running_regression_loss[step] += regression_loss
        running_regression_loss_1h[step] += regression_loss_1h
        running_segmentation_loss[step] += segmentation_loss
        running_confusion_matrix[step] += batch_cm
        
    optimizer.step()  # Update the weights after the loop
    
    del inputs0, inputs1, outputs0, outputs1, split_targets, loss0, loss1, loss, regression_loss, 
    del regression_loss_1h, segmentation_loss, noisy_images, images, pairs, filters, segment_measurements_fcn
    del point_measurements_fcn_train_targets
    torch.cuda.empty_cache()

  if epoch > 0:
    for step in steps:
      regression_loss = running_regression_loss[step] / len(loader)
      regression_loss_1h = running_regression_loss_1h[step] / len(loader)
      segmentation_loss = running_segmentation_loss[step] / len(loader)
      losses[step].append((epoch, regression_loss, regression_loss_1h, segmentation_loss, running_confusion_matrix[step]))

      print(f'{step}, Regression Loss: {regression_loss:.4f}, Regression Loss 1h: {regression_loss_1h:.4f}, Segmentation Loss:{segmentation_loss:.4f}' )
      print("Train Confusion Matrix:")
      print(running_confusion_matrix[step])
      accuracy, csi, sensitivity, specificity, false_alarm_ratio = compute_metrics(running_confusion_matrix[step])
      print(f'Accuracy: {accuracy:.4f}, CSI: {csi:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, False Alarm Ratio: {false_alarm_ratio:.4f}')
      print('\n')
  print('epoch duration :', time.time() - t)

  if (epoch % save_every == 0 or \
    epoch == last_epoch):
    print("saving step")
    checkpoint = { 
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        # 'scheduler': scheduler.state_dict(),
        'train_losses': losses,
        }
    torch.save(checkpoint, path)  
    

epoch n° 54 

eval_opportunity_cost_spat, Regression Loss: 0.0345, Regression Loss 1h: 260.9757, Segmentation Loss:0.1268
Train Confusion Matrix:
[[3116555   81305]
 [ 111433  530707]]
Accuracy: 0.9498, CSI: 0.7336, Sensitivity: 0.8265, Specificity: 0.9746, False Alarm Ratio: 0.1328


eval_added_value_full_spat, Regression Loss: 0.0316, Regression Loss 1h: 264.1949, Segmentation Loss:0.1184
Train Confusion Matrix:
[[3120426   77434]
 [ 104464  537676]]
Accuracy: 0.9526, CSI: 0.7472, Sensitivity: 0.8373, Specificity: 0.9758, False Alarm Ratio: 0.1259


train, Regression Loss: 0.3145, Regression Loss 1h: 2646.2965, Segmentation Loss:1.1649
Train Confusion Matrix:
[[3120604   73750]
 [ 106130  539516]]
Accuracy: 0.9532, CSI: 0.7500, Sensitivity: 0.8356, Specificity: 0.9769, False Alarm Ratio: 0.1203


epoch duration : 1051.930097103119
saving step
epoch n° 55 

eval_opportunity_cost_spat, Regression Loss: 0.0342, Regression Loss 1h: 264.0845, Segmentation Loss:0.1251
Train Confusion Matri